In [1]:
import os
from glob import glob
import numpy as np
import json

import cv2
from tqdm.notebook import tqdm

import ffmpeg_utils
from augment_script import process_video, process_images
from augment import Augmentations
from writer import COCO_writer
from bbox_utils import get_scale_ratio, resize_by_max_side

### Preprocess effects

In [2]:
e_type = 'fire'
e_type = 'smoke'

def get_effect_paths(e_type):
    assert e_type in ['fire', 'smoke'], "Unsupported"
    raw_png_effects_path  = f'effects/raw_{e_type}_images'
    prep_png_effects_path = f'effects/prep_{e_type}_images'
    raw_mov_effects_path  = f'effects/raw_{e_type}_vid'
    prep_mov_effects_path = f'effects/prep_{e_type}_vid'
    return (raw_png_effects_path, prep_png_effects_path,
            raw_mov_effects_path, prep_mov_effects_path)

(raw_png_effects_path, prep_png_effects_path,
 raw_mov_effects_path, prep_mov_effects_path) = get_effect_paths(e_type)


vid_out_path = 'output/out.mp4'
annot_out_path = 'output/annotations/instances_default.json'

# Create folders
for path in [raw_png_effects_path, prep_png_effects_path,
             raw_mov_effects_path, prep_mov_effects_path,
             os.path.split(vid_out_path)[0],
             os.path.split(annot_out_path)[0]]:
    os.makedirs(path, exist_ok=True)

In [3]:
# Rename and trim empty pixels
# raw_png_effects_path = 'effects/non_funny'
# prep_png_effects_path = 'effects/prep_non_funny'
# e_type = 'non-funny'
# for i, effect_path in enumerate(glob(os.path.join(raw_png_effects_path, '*.png'))):
#     e_img = cv2.imread(effect_path, cv2.IMREAD_UNCHANGED)
#     # Trim empty pixels
#     y, x = e_img[:, :, 3].nonzero()
#     minx, miny = np.min(x), np.min(y)
#     maxx, maxy = np.max(x), np.max(y)
#     e_img = e_img[miny:maxy, minx:maxx]
#     # Resize to 512px
#     scale_ratio = get_scale_ratio(e_img, 512)
#     e_img = resize_by_max_side(e_img, scale_ratio)
#     os.makedirs(prep_png_effects_path, exist_ok=True)
#     cv2.imwrite(os.path.join(prep_png_effects_path, f'{e_type}-{i}.png'), e_img)

In [4]:
# Rename videos
# for i, mov_path in enumerate(glob(os.path.join(raw_mov_effects_path, '*.mov'))):
#     print(mov_path)
#     path = os.path.split(mov_path)[0]
#     os.rename(mov_path, os.path.join(path, f'{e_type}-{i}.mov'))

In [5]:
# Resize and extract alpha videos

# for input_path in glob(os.path.join(raw_mov_effects_path, '*.mov')):
#     input_path = os.path.abspath(input_path)
#     filename = os.path.splitext(os.path.split(input_path)[1])[0]
#     out_path = os.path.join(prep_mov_effects_path, filename + '.webm')
#     out_path = os.path.abspath(out_path)
#     ffmpeg_utils.convert_mov2webm(input_path, out_path, y=True)
#     ffmpeg_utils.extract_alpha(out_path, y=True)

In [6]:
e_paths = get_effect_paths('fire')
prep_e_png_fire_path, prep_e_mov_fire_path = e_paths[1], e_paths[3]
e_paths = get_effect_paths('smoke')
prep_e_png_smoke_path, prep_e_mov_smoke_path = e_paths[1], e_paths[3]

In [7]:
e_png_fire = glob(os.path.join(prep_e_png_fire_path, '*.png'))
e_mov_fire = glob(os.path.join(prep_e_mov_fire_path, '*.webm'))
e_png_smoke = glob(os.path.join(prep_e_png_smoke_path, '*.png'))
e_mov_smoke = glob(os.path.join(prep_e_mov_smoke_path, '*.webm'))
e_png_fire, e_mov_fire, e_png_smoke, e_mov_smoke

(['effects/prep_fire_images\\fire-0.png',
  'effects/prep_fire_images\\fire-1.png',
  'effects/prep_fire_images\\fire-2.png'],
 ['effects/prep_fire_vid\\fire-0.webm',
  'effects/prep_fire_vid\\fire-1.webm',
  'effects/prep_fire_vid\\fire-2.webm',
  'effects/prep_fire_vid\\fire-3.webm',
  'effects/prep_fire_vid\\fire-4.webm',
  'effects/prep_fire_vid\\fire-5.webm',
  'effects/prep_fire_vid\\fire-6.webm',
  'effects/prep_fire_vid\\fire-7.webm'],
 ['effects/prep_smoke_images\\smoke-0.png',
  'effects/prep_smoke_images\\smoke-1.png',
  'effects/prep_smoke_images\\smoke-2.png',
  'effects/prep_smoke_images\\smoke-3.png',
  'effects/prep_smoke_images\\smoke-4.png',
  'effects/prep_smoke_images\\smoke-5.png'],
 ['effects/prep_smoke_vid\\smoke-0.webm',
  'effects/prep_smoke_vid\\smoke-1.webm'])

### Augment

In [8]:
source_videos = glob('source_videos/*')
source_videos

['source_videos\\2020-06-23_16-40-40.mp4',
 'source_videos\\stream_OV1_2020-08-01_09_58_11.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_09_58_50.ts.mp4']

In [9]:
# COCO_writer
coco_writer = COCO_writer([
    {
        'name': 'Fire',
        'supercategory': '',
        'id': 2,
    },
    {
        'name': 'Smoke',
        'supercategory': '',
        'id': 4,
    },
])


# Augmentations
fire_augment = Augmentations(
    e_png_fire,
    [],
#     e_mov_fire,
    config_path='augment_config.yaml',
    do_resize=True,
    do_flip=True,
    do_rotate=True,
    debug_level=0,
    min_n_objects=1,
    max_n_objects=1,
    use_alpha=True,
    min_duration=1,
    max_duration=2,
)

# smoke_augment = Augmentations(
#     e_png_smoke,
#     e_mov_smoke,
#     config_path='augment_config.yaml',
#     do_resize=True,
#     do_flip=True,
#     do_rotate=False,
#     do_contrast=False,
#     debug_level=2,
#     ck_start=2,
#     ck_range=15,
#     max_n_objects=3,
#     use_alpha=True
# )

augmentations = [
    fire_augment,
#     smoke_augment
]

# process_video(source_videos[0], augmentations, vid_out_path, coco_writer, show_debug=True)

# # Write annotations.
# os.makedirs(os.path.split(annot_out_path)[0], exist_ok=True)
# coco_writer.write_result(annot_out_path)

In [10]:
%load_ext line_profiler
%load_ext memory_profiler

In [14]:
from augment import Augmentations
from augment_script import _process_image
from bbox_utils import (convert_xywh_xyxy, convert_xyxy_xywh,
                        resize, rotate, flip, gamma_correction,
                        blur_contour)

augment = Augmentations.augment
merge_images = Augmentations.merge_images

In [15]:
%lprun -f blur_contour process_video(source_videos[0], augmentations, vid_out_path, coco_writer, show_debug=False)

  1%|▌                                                                             | 101/15004 [00:08<21:04, 11.79it/s]

Exited.


In [ ]:
import sys
sys.getsizeof(np.array(images))

In [ ]:
sys.getsizeof(np.array(images)) / (1024 ** 2)

In [12]:
images = glob(os.path.join('ufa/images', f'*.jpg'))
out_path = 'output/test'
show_debug, write_debug = False, False
%lprun -f process_images process_images(images, augmentations, out_path, coco_writer, show_debug, write_debug, n_workers=None)

 13%|██████████▌                                                                    | 178/1335 [00:14<01:36, 11.99it/s]

Exited.


In [12]:
a = np.random.normal(loc=1, scale=0.05, size=10000)
b = np.random.normal(loc=0, scale=2.5, size=10000)
print(a.min(), a.max(), a.mean())
print(b.min(), b.max(), b.mean())

0.8101393811714315 1.2096084941876668 1.0002359061872779
-10.671825048672561 10.881186819007691 -0.03747667585126547


In [15]:
a = np.random.random((480, 720, 4))
a[:, :, 3:].shape

(480, 720, 1)

In [16]:
1335 / 101

13.217821782178218

In [13]:
images = glob('effects/prep_non_funny/*.png')

In [33]:
cv2.destroyAllWindows()

#### TODO


##### General
* <s>move everything to config file</s>
* написать loop чтобы обработать все видосы

##### Preparation
* <s>remove clear pixels</s>
* remove clear pixels from video

##### Augmentation
* <s>cover all image</s>
* <s>change sizes</s>
* <s>flip</s>
* <s>add animations</s>
* <s>proper resizing</s>
* <s>make an offset point a down center point of the effect image.</s>
* <s>fix merging with video - fix: Color keying</s>
* <s>поиграться с color keying'ом, чтобы более плавные переходы в нём были</s>
* <s>change angle</s>
* <s>contrast and brightness</s>
* <s>gamma correction</s>
* <s>find new effects, add smoke</s>
* warp perspective

##### Annotation
* <s>add dynamic bboxes for videos</s>
* <s>fix bboxes for some videos</s>
* <s>annotate all videos</s>
* <s>annotation for images</s>
* <s>scaling bboxes</s>
* <s>rotating bboxes</s>
* <s>Write to COCO</s>
* <s>Get class from annotation, not by fixing it for Augmentation object</s>
* разметить эффекты не боксом, а полигоном, чтобы можно было нормально поворачивать эффект и при этом иметь нормальный бокс



## Working with videos

#### Scale down with keeping alpha channel
```shell
ffmpeg -i in.mov -filter:v scale=720:-1 -c:v qtrle out.mov
```

#### `.mov` to `.webm` keeping  alpha channel

```shell
ffmpeg -i in.mov -c:v libvpx -pix_fmt yuva420p -auto-alt-ref 0 out.webm
```

#### Extract alpha channel from `.webm`

```shell
ffmpeg -vcodec libvpx -i in.webm -vf alphaextract -y out.mp4
```


#### Extract alpha channel from `.mov`

```shell
ffmpeg -i in.mov -vf alphaextract,format=yuv420p out.mp4
```

#### Compress resulting video
```shell
ffmpeg -i out.mp4 -vcodec h264 -b:v 1000k -acodec mp2 compressed_out.mp4 -y
```

#### Get first `n` frames
```shell
ffmpeg -i in.mp4 -frames:v 249 -c copy out.mp4
```

#### Cut video from `ss` till `t`
```shell
ffmpeg -ss 00:00:8.25 -i in.mp4 -t 00:00:8.25 out.mp4                                                               
```

#### Merge video with alpha channel in `.mov` file
```shell
ffmpeg -i in.mp4 -i alpha_in.mp4 -filter_complex [0][1]alphamerge -c:v qtrle out.mov
```